# Introduction

I really want to get the info about streaming data to do some analysis along with the box office info. I'm curious whether movies that did well at the box office are more likely to be available for rent or buy versus subscription, and whether they are more likely to be on multiple platforms. I think this is important to figure out if the same things that make a movie successful at the box office will also make it successful in post-release streaming.

The problem is, this data isn't easy to find. The Numbers has a paid data set that I don't want to pay for. So does JustWatch.

I checked out https://api.watchmode.com/, which has a free option.

In [53]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

## Watchmode API

In [61]:
# Credit to https://stackoverflow.com/questions/4028904/
#    how-to-get-the-home-directory-in-python for how to get the home directory
def get_keys(path):
    from os.path import expanduser
    home = expanduser("~")
    with open(path.replace('~', home)) as f:
        return json.load(f)

keys = get_keys("~/.secret/watchmode_api.json")
api_key = keys['api_key']
len(api_key)

40

In [62]:
# Test out the new API with key and make sure we get a good response
url = 'https://api.watchmode.com/v1/list-titles'

url_params = {
                'apiKey': api_key,
                'types': 'movie',
                'regions': 'US',
                'source_types': 'sub',
                'page': '1'
             }
# source_types: sub, rent, buy, free, tve
# Starting with 'sub' for subscription

response = requests.get(url, params=url_params)
response.status_code

200

In [64]:
# Load the resonse into a dictionary and check out the keys
resp_dict = json.loads(response.text)

resp_dict.keys()

dict_keys(['titles', 'page', 'total_results', 'total_pages'])

In [66]:
# How many results did we get?
print(f"Total results available: {resp_dict['total_results']}")
print(f"Total pages available: {resp_dict['total_pages']}")

Total results available: 17783
Total pages available: 72


In [68]:
resp_dict['titles'][0]

{'id': 11,
 'title': '!Women Art Revolution',
 'year': 2010,
 'imdb_id': 'tt1699720',
 'tmdb_id': 55245,
 'tmdb_type': 'movie',
 'type': 'movie'}

## Attempt to scrape from JustWatch

Justwatch has infinite scroll, and the info I want isn't on the initial page, so I would need to loop through the pages. I was able to get some of the pages 

In [44]:
body = {"fields":["cinema_release_date","full_path","full_paths","id","localized_release_date","object_type","poster","scoring","title","tmdb_popularity","backdrops"],"content_types":["movie"],"providers":["nfx","amp","dnp","fuv","atp","itu","hlu","hbm","pct","pcp","amz","ply","yot","ytr","hbn","sho"],"sort_by":"release_year","enable_provider_filter":False,"monetization_types":[],"page":3,"page_size":30,"matching_offers_only":True}
body_json = json.dumps(body)
body_json

'{"fields": ["cinema_release_date", "full_path", "full_paths", "id", "localized_release_date", "object_type", "poster", "scoring", "title", "tmdb_popularity", "backdrops"], "content_types": ["movie"], "providers": ["nfx", "amp", "dnp", "fuv", "atp", "itu", "hlu", "hbm", "pct", "pcp", "amz", "ply", "yot", "ytr", "hbn", "sho"], "sort_by": "release_year", "enable_provider_filter": false, "monetization_types": [], "page": 3, "page_size": 30, "matching_offers_only": true}'

In [45]:
html_page = requests.get("""https://apis.justwatch.com/content/titles/en_US/popular"""
                        , data=body_json)
soup = BeautifulSoup(html_page.content)

In [46]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="initial-scale=1, minimum-scale=1, width=device-width" name="viewport"/>\n  <title>\n   Error 400 (Bad Request)!!1\n  </title>\n  <style>\n   *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100%

OK, so I managed to find the API call used to get more results, but I don't think it will allow me to use it. Which makes sense since I'm making the call externally without a request header :-p

However, the info saved in devtools actually has the links for the returned movies, so I can definitely parse those out of the HAR file from devtools and them loop through the sites to get the actual info. I have to manually scroll, but it's pretty easy to do. The site only loads 66 pages of 30 results total, which is only 1980 results at a time, so I have to figure out how to filter my results to be under that limit.

Without specifying a release date in the filter, I get results that haven't been released yet... going to come back to that. Started with filtering by release year, but got too many movies. Tried also filtering by the streaming service for the most popular ones but, still too many. Finally, selected only the services which are subscription only - no rentals, and monetization is subscription. Success! Will keep trying this and logging the files with the criteria.

### Filter criteria used and results file tracking

|URL|Release Year|Services|Monetization|Results|Saved File Name|
|---|---|---|---|---|---|
|//www.justwatch.com/us/movies<br>?monetization_types=flatrate<br>&providers=amp,atp,dnp,hbm,hlu,nfx<br>&release_year_from=2020<br>&release_year_until=2020<br>&sort_by=release_year  |2020|amp,atp,dnp,hbm,hlu,nfx|Subscription|1602|JustWatch_released2020_AvailBySubscription.har|
|//www.justwatch.com/us/movies<br>?monetization_types=flatrate<br>&providers=atp,dnp,hbm,hlu,nfx<br>&release_year_from=2019<br>&release_year_until=2019<br>&sort_by=release_year |2019|atp,dnp,hbm,hlu,nfx|Subscription|941|JustWatch_released2019_AvailBySubscription_NoPrime.har|
|//www.justwatch.com/us/movies<br>?monetization_types=flatrate<br>&providers=amp<br>&release_year_from=2019<br>&release_year_until=2019<br>&sort_by=release_year|2019|amp|Subscription|1800|JustWatch_released2019_AvailBySubscription_PrimeOnly.har|
|---|---|---|---|---|---|
|---|---|---|---|---|---|
|---|---|---|---|---|---|
|---|---|---|---|---|---|
|---|---|---|---|---|---|
|---|---|---|---|---|---|


These are the popular services that don't have rentals or buy options; you get what you get with your subscription:

- amp = Amazon Prime Video
- atp = Apple TV Plus
- dnp = Disney Plus
- hbm = HBO Max
- hlu = Hulu
- nfx = Netflix

### Parsing the HAR files to get paths

Before getting too far with the HAR files, I want to make sure I'll be able to parse the paths I need from them and make a script to scrape the info from the individual page URLs.

Credit to https://www.accordbox.com/blog/how-crawl-infinite-scrolling-pages-using-python/ for the instructions on how to web scrape a site with infinite scrolling, although it didn't work. Accessed 3/14/2021.

Credit to https://stackoverflow.com/questions/4028904/how-to-get-the-home-directory-in-python for how to get the home directory in Python. Accessed 3/14/2021.